DEEP REINFORCEMENT LEARNING EXPLAINED - 15 - 16 - 17
# **Deep Q-Network (DQN)**

OpenAI Pong

In [1]:
import gym
import gym.spaces

DEFAULT_ENV_NAME = "PongNoFrameskip-v4" 
test_env = gym.make(DEFAULT_ENV_NAME)
print(test_env.action_space.n)

6


In [2]:
print(test_env.unwrapped.get_action_meanings())

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']


In [3]:
print(test_env.observation_space.shape)

(210, 160, 3)



Type of hardware accelerator provided by Colab

In [4]:
!nvidia-smi 

Tue Apr 20 22:32:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.56       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 165...  Off  | 00000000:01:00.0  On |                  N/A |
| 60%   36C    P8    12W / 100W |   1188MiB /  3908MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import warnings
warnings.filterwarnings('ignore')

## OpenAI Gym Wrappers

In [7]:
# Taken from 
# https://github.com/PacktPublishing/Deep-Reinforcement-Learning-Hands-On/blob/master/Chapter06/lib/wrappers.py

import cv2
import numpy as np
import collections

class FireResetEnv(gym.Wrapper):
    def __init__(self, env=None):
        super(FireResetEnv, self).__init__(env)
        assert env.unwrapped.get_action_meanings()[1] == 'FIRE'
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def step(self, action):
        return self.env.step(action)

    def reset(self):
        self.env.reset()
        obs, _, done, _ = self.env.step(1)
        if done:
            self.env.reset()
        obs, _, done, _ = self.env.step(2)
        if done:
            self.env.reset()
        return obs

class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env=None, skip=4):
        super(MaxAndSkipEnv, self).__init__(env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = collections.deque(maxlen=2)
        self._skip = skip

    def step(self, action):
        total_reward = 0.0
        done = None
        for _ in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            self._obs_buffer.append(obs)
            total_reward += reward
            if done:
                break
        max_frame = np.max(np.stack(self._obs_buffer), axis=0)
        return max_frame, total_reward, done, info

    def reset(self):
        self._obs_buffer.clear()
        obs = self.env.reset()
        self._obs_buffer.append(obs)
        return obs


class ProcessFrame84(gym.ObservationWrapper):
    def __init__(self, env=None):
        super(ProcessFrame84, self).__init__(env)
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)

    def observation(self, obs):
        return ProcessFrame84.process(obs)

    @staticmethod
    def process(frame):
        if frame.size == 210 * 160 * 3:
            img = np.reshape(frame, [210, 160, 3]).astype(np.float32)
        elif frame.size == 250 * 160 * 3:
            img = np.reshape(frame, [250, 160, 3]).astype(np.float32)
        else:
            assert False, "Unknown resolution."
        img = img[:, :, 0] * 0.299 + img[:, :, 1] * 0.587 + img[:, :, 2] * 0.114
        resized_screen = cv2.resize(img, (84, 110), interpolation=cv2.INTER_AREA)
        x_t = resized_screen[18:102, :]
        x_t = np.reshape(x_t, [84, 84, 1])
        return x_t.astype(np.uint8)


class BufferWrapper(gym.ObservationWrapper):
    def __init__(self, env, n_steps, dtype=np.float32):
        super(BufferWrapper, self).__init__(env)
        self.dtype = dtype
        old_space = env.observation_space
        self.observation_space = gym.spaces.Box(old_space.low.repeat(n_steps, axis=0),
                                                old_space.high.repeat(n_steps, axis=0), dtype=dtype)

    def reset(self):
        self.buffer = np.zeros_like(self.observation_space.low, dtype=self.dtype)
        return self.observation(self.env.reset())

    def observation(self, observation):
        self.buffer[:-1] = self.buffer[1:]
        self.buffer[-1] = observation
        return self.buffer


class ImageToPyTorch(gym.ObservationWrapper):
    def __init__(self, env):
        super(ImageToPyTorch, self).__init__(env)
        old_shape = self.observation_space.shape
        self.observation_space = gym.spaces.Box(low=0.0, high=1.0, shape=(old_shape[-1], 
                                old_shape[0], old_shape[1]), dtype=np.float32)

    def observation(self, observation):
        return np.moveaxis(observation, 2, 0)


class ScaledFloatFrame(gym.ObservationWrapper):
    def observation(self, obs):
        return np.array(obs).astype(np.float32) / 255.0

def make_env(env_name):
    env = gym.make(env_name)
    env = MaxAndSkipEnv(env)
    env = FireResetEnv(env)
    env = ProcessFrame84(env)
    env = ImageToPyTorch(env)
    env = BufferWrapper(env, 4)
    return ScaledFloatFrame(env)

## The DQN model


In [8]:
import torch
import torch.nn as nn        # Pytorch neural network package
import torch.optim as optim  # Pytorch optimization package

device = torch.device("cuda")

In [9]:
# Taken from 
# https://github.com/PacktPublishing/Deep-Reinforcement-Learning-Hands-On/blob/master/Chapter06/lib/dqn_model.py

import numpy as np

class DQN(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(DQN, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )

        conv_out_size = self._get_conv_out(input_shape)
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )

    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))

    def forward(self, x):
        conv_out = self.conv(x).view(x.size()[0], -1)
        return self.fc(conv_out)

In [10]:
test_env = make_env(DEFAULT_ENV_NAME)
test_net = DQN(test_env.observation_space.shape, test_env.action_space.n).to(device)
print(test_net)

DQN(
  (conv): Sequential(
    (0): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=3136, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=6, bias=True)
  )
)


## Training

Load Tensorboard extension

In [12]:
from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard

Import required modules and define the hyperparameters

In [13]:
import time
import numpy as np
import collections


MEAN_REWARD_BOUND = 19.0           

gamma = 0.99                   
batch_size = 32                
replay_size = 10000            
learning_rate = 1e-4           
sync_target_frames = 1000      
replay_start_size = 10000      

eps_start=1.0
eps_decay=.999985
eps_min=0.02

Experience replay buffer

In [14]:
Experience = collections.namedtuple('Experience', field_names=['state', 'action', 'reward', 'done', 'new_state'])

class ExperienceReplay:
    def __init__(self, capacity):
        self.buffer = collections.deque(maxlen=capacity)

    def __len__(self):
        return len(self.buffer)

    def append(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        indices = np.random.choice(len(self.buffer), batch_size, replace=False)
        states, actions, rewards, dones, next_states = zip(*[self.buffer[idx] for idx in indices])
        return np.array(states), np.array(actions), np.array(rewards, dtype=np.float32), \
               np.array(dones, dtype=np.uint8), np.array(next_states)


Agent

In [15]:
class Agent:
    def __init__(self, env, exp_buffer):
        self.env = env
        self.exp_buffer = exp_buffer
        self._reset()

    def _reset(self):
        self.state = env.reset()
        self.total_reward = 0.0

    def play_step(self, net, epsilon=0.0, device="cpu"):

        done_reward = None
        if np.random.random() < epsilon:
            action = env.action_space.sample()
        else:
            state_a = np.array([self.state], copy=False)
            state_v = torch.tensor(state_a).to(device)
            q_vals_v = net(state_v)
            _, act_v = torch.max(q_vals_v, dim=1)
            action = int(act_v.item())

        new_state, reward, is_done, _ = self.env.step(action)
        self.total_reward += reward

        exp = Experience(self.state, action, reward, is_done, new_state)
        self.exp_buffer.append(exp)
        self.state = new_state
        if is_done:
            done_reward = self.total_reward
            self._reset()
        return done_reward


In [16]:
import datetime
print(">>>Training starts at ",datetime.datetime.now())

>>>Training starts at  2021-04-20 22:34:53.354436


Main training loop

In [17]:
env = make_env(DEFAULT_ENV_NAME)

net = DQN(env.observation_space.shape, env.action_space.n).to(device)
target_net = DQN(env.observation_space.shape, env.action_space.n).to(device)
writer = SummaryWriter(comment="-" + DEFAULT_ENV_NAME)
 
buffer = ExperienceReplay(replay_size)
agent = Agent(env, buffer)

epsilon = eps_start

optimizer = optim.Adam(net.parameters(), lr=learning_rate)
total_rewards = []
frame_idx = 0  

best_mean_reward = None

while True:
        frame_idx += 1
        epsilon = max(epsilon*eps_decay, eps_min)

        reward = agent.play_step(net, epsilon, device=device)
        if reward is not None:
            total_rewards.append(reward)

            mean_reward = np.mean(total_rewards[-100:])

            print("%d:  %d games, mean reward %.3f, (epsilon %.2f)" % (
                frame_idx, len(total_rewards), mean_reward, epsilon))
            
            writer.add_scalar("epsilon", epsilon, frame_idx)
            writer.add_scalar("reward_100", mean_reward, frame_idx)
            writer.add_scalar("reward", reward, frame_idx)

            if best_mean_reward is None or best_mean_reward < mean_reward:
                torch.save(net.state_dict(), DEFAULT_ENV_NAME + "-best.dat")
                best_mean_reward = mean_reward
                if best_mean_reward is not None:
                    print("Best mean reward updated %.3f" % (best_mean_reward))

            if mean_reward > MEAN_REWARD_BOUND:
                print("Solved in %d frames!" % frame_idx)
                break

        if len(buffer) < replay_start_size:
            continue

        batch = buffer.sample(batch_size)
        states, actions, rewards, dones, next_states = batch

        states_v = torch.tensor(states).to(device)
        next_states_v = torch.tensor(next_states).to(device)
        actions_v = torch.tensor(actions).to(device)
        rewards_v = torch.tensor(rewards).to(device)
        done_mask = torch.ByteTensor(dones).to(device)

        state_action_values = net(states_v).gather(1, actions_v.unsqueeze(-1)).squeeze(-1)

        next_state_values = target_net(next_states_v).max(1)[0]

        next_state_values[done_mask] = 0.0

        next_state_values = next_state_values.detach()

        expected_state_action_values = next_state_values * gamma + rewards_v

        loss_t = nn.MSELoss()(state_action_values, expected_state_action_values)

        optimizer.zero_grad()
        loss_t.backward()
        optimizer.step()

        if frame_idx % sync_target_frames == 0:
            target_net.load_state_dict(net.state_dict())
       
writer.close()

869:  1 games, mean reward -20.000, (epsilon 0.99)
Best mean reward updated -20.000
1751:  2 games, mean reward -20.500, (epsilon 0.97)
2757:  3 games, mean reward -20.333, (epsilon 0.96)
3598:  4 games, mean reward -20.250, (epsilon 0.95)
4360:  5 games, mean reward -20.400, (epsilon 0.94)
5230:  6 games, mean reward -20.333, (epsilon 0.92)
6190:  7 games, mean reward -20.286, (epsilon 0.91)
6952:  8 games, mean reward -20.375, (epsilon 0.90)
7900:  9 games, mean reward -20.333, (epsilon 0.89)
8662:  10 games, mean reward -20.400, (epsilon 0.88)
9484:  11 games, mean reward -20.455, (epsilon 0.87)
10306:  12 games, mean reward -20.500, (epsilon 0.86)
11216:  13 games, mean reward -20.538, (epsilon 0.85)
12179:  14 games, mean reward -20.571, (epsilon 0.83)
13158:  15 games, mean reward -20.533, (epsilon 0.82)
14135:  16 games, mean reward -20.438, (epsilon 0.81)
15270:  17 games, mean reward -20.353, (epsilon 0.80)
16366:  18 games, mean reward -20.333, (epsilon 0.78)
17320:  19 games

186428:  115 games, mean reward -16.190, (epsilon 0.06)
Best mean reward updated -16.190
189336:  116 games, mean reward -15.910, (epsilon 0.06)
Best mean reward updated -15.910
191818:  117 games, mean reward -15.620, (epsilon 0.06)
Best mean reward updated -15.620
193976:  118 games, mean reward -15.270, (epsilon 0.05)
Best mean reward updated -15.270
196770:  119 games, mean reward -14.970, (epsilon 0.05)
Best mean reward updated -14.970
199803:  120 games, mean reward -14.710, (epsilon 0.05)
Best mean reward updated -14.710
202334:  121 games, mean reward -14.420, (epsilon 0.05)
Best mean reward updated -14.420
204472:  122 games, mean reward -14.050, (epsilon 0.05)
Best mean reward updated -14.050
207042:  123 games, mean reward -13.720, (epsilon 0.04)
Best mean reward updated -13.720
210108:  124 games, mean reward -13.480, (epsilon 0.04)
Best mean reward updated -13.480
212050:  125 games, mean reward -13.100, (epsilon 0.04)
Best mean reward updated -13.100
214573:  126 games, m

388178:  210 games, mean reward 14.840, (epsilon 0.02)
Best mean reward updated 14.840
390077:  211 games, mean reward 15.140, (epsilon 0.02)
Best mean reward updated 15.140
391990:  212 games, mean reward 15.400, (epsilon 0.02)
Best mean reward updated 15.400
393750:  213 games, mean reward 15.520, (epsilon 0.02)
Best mean reward updated 15.520
395745:  214 games, mean reward 15.600, (epsilon 0.02)
Best mean reward updated 15.600
397647:  215 games, mean reward 15.740, (epsilon 0.02)
Best mean reward updated 15.740
399287:  216 games, mean reward 15.860, (epsilon 0.02)
Best mean reward updated 15.860
400995:  217 games, mean reward 15.950, (epsilon 0.02)
Best mean reward updated 15.950
403178:  218 games, mean reward 15.940, (epsilon 0.02)
405120:  219 games, mean reward 16.020, (epsilon 0.02)
Best mean reward updated 16.020
407541:  220 games, mean reward 16.080, (epsilon 0.02)
Best mean reward updated 16.080
409471:  221 games, mean reward 16.170, (epsilon 0.02)
Best mean reward upd

613060:  330 games, mean reward 18.250, (epsilon 0.02)
Best mean reward updated 18.250
614908:  331 games, mean reward 18.250, (epsilon 0.02)
616825:  332 games, mean reward 18.250, (epsilon 0.02)
618775:  333 games, mean reward 18.250, (epsilon 0.02)
620559:  334 games, mean reward 18.270, (epsilon 0.02)
Best mean reward updated 18.270
622475:  335 games, mean reward 18.260, (epsilon 0.02)
624327:  336 games, mean reward 18.250, (epsilon 0.02)
626270:  337 games, mean reward 18.240, (epsilon 0.02)
628216:  338 games, mean reward 18.250, (epsilon 0.02)
629945:  339 games, mean reward 18.260, (epsilon 0.02)
631933:  340 games, mean reward 18.280, (epsilon 0.02)
Best mean reward updated 18.280
633828:  341 games, mean reward 18.290, (epsilon 0.02)
Best mean reward updated 18.290
635557:  342 games, mean reward 18.300, (epsilon 0.02)
Best mean reward updated 18.300
637262:  343 games, mean reward 18.290, (epsilon 0.02)
639153:  344 games, mean reward 18.330, (epsilon 0.02)
Best mean rewar

864117:  465 games, mean reward 18.690, (epsilon 0.02)
866088:  466 games, mean reward 18.730, (epsilon 0.02)
Best mean reward updated 18.730
868125:  467 games, mean reward 18.780, (epsilon 0.02)
Best mean reward updated 18.780
869822:  468 games, mean reward 18.800, (epsilon 0.02)
Best mean reward updated 18.800
871750:  469 games, mean reward 18.790, (epsilon 0.02)
873441:  470 games, mean reward 18.840, (epsilon 0.02)
Best mean reward updated 18.840
875263:  471 games, mean reward 18.830, (epsilon 0.02)
876929:  472 games, mean reward 18.860, (epsilon 0.02)
Best mean reward updated 18.860
878929:  473 games, mean reward 18.900, (epsilon 0.02)
Best mean reward updated 18.900
880626:  474 games, mean reward 18.910, (epsilon 0.02)
Best mean reward updated 18.910
882376:  475 games, mean reward 18.950, (epsilon 0.02)
Best mean reward updated 18.950
884436:  476 games, mean reward 18.920, (epsilon 0.02)
886335:  477 games, mean reward 18.890, (epsilon 0.02)
888083:  478 games, mean rewa

1129619:  609 games, mean reward 18.880, (epsilon 0.02)
1131495:  610 games, mean reward 18.860, (epsilon 0.02)
1133401:  611 games, mean reward 18.910, (epsilon 0.02)
1135097:  612 games, mean reward 18.980, (epsilon 0.02)
Best mean reward updated 18.980
1137080:  613 games, mean reward 18.970, (epsilon 0.02)
1138962:  614 games, mean reward 18.960, (epsilon 0.02)
1141043:  615 games, mean reward 18.940, (epsilon 0.02)
1142694:  616 games, mean reward 18.950, (epsilon 0.02)
1144387:  617 games, mean reward 18.980, (epsilon 0.02)
1146335:  618 games, mean reward 18.970, (epsilon 0.02)
1148222:  619 games, mean reward 18.960, (epsilon 0.02)
1150450:  620 games, mean reward 18.950, (epsilon 0.02)
1152514:  621 games, mean reward 18.930, (epsilon 0.02)
1154250:  622 games, mean reward 18.930, (epsilon 0.02)
1156113:  623 games, mean reward 18.900, (epsilon 0.02)
1157884:  624 games, mean reward 18.870, (epsilon 0.02)
1159777:  625 games, mean reward 18.860, (epsilon 0.02)
1161658:  626 ga

In [18]:
print(">>>Training ends at ",datetime.datetime.now())

>>>Training ends at  2021-04-21 01:05:43.929660


Performance

In [19]:
tensorboard  --logdir=runs

## Using the model

In [20]:
import gym
import time
import numpy as np

import torch

import collections

DEFAULT_ENV_NAME = "PongNoFrameskip-v4"
FPS = 25

Tunning the image rendering in colab


In [28]:
# Taken from 
# https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7

# !apt-get install -y xvfb x11-utils

# !pip install pyvirtualdisplay==0.2.* \
#             PyOpenGL==3.1.* \
#             PyOpenGL-accelerate==3.1.*

# !pip install gym[box2d]==0.17.*

import pyvirtualdisplay

_display = pyvirtualdisplay.Display(visible=False, size=(1400, 900))
_ = _display.start()

In [1]:
# Taken (partially) from 
# https://github.com/PacktPublishing/Deep-Reinforcement-Learning-Hands-On/blob/master/Chapter06/03_dqn_play.py


model='PongNoFrameskip-v4-best.dat'
record_folder="test"  
visualize=True

env = make_env(DEFAULT_ENV_NAME)
if record_folder:
        env = gym.wrappers.Monitor(env, record_folder, force=True)
net = DQN(env.observation_space.shape, env.action_space.n)
net.load_state_dict(torch.load(model, map_location=lambda storage, loc: storage))

state = env.reset()
total_reward = 0.0

while True:
        start_ts = time.time()
        if visualize:
            env.render()
        state_v = torch.tensor(np.array([state], copy=False))
        q_vals = net(state_v).data.numpy()[0]
        action = np.argmax(q_vals)
        
        state, reward, done, _ = env.step(action)
        total_reward += reward
        if done:
            break
        if visualize:
            delta = 1/FPS - (time.time() - start_ts)
            if delta > 0:
                time.sleep(delta)
print("Total reward: %.2f" % total_reward)

if record_folder:
        env.close()

NameError: name 'make_env' is not defined